In [ ]:
# Script to visualize the snow cover duration for every 250-meter elevation zone,
# to calculate the mean SCD,
# and to visualize the mean SCD above 400 meters

In [ ]:
# Before running this script: 
# 1. Use the script Download_MODIS.ipynb to download the needed MODIS Snow Cover Terra and Aqua data
# 2. Use the script ReplaceMissing_MODIS.ipynb to search for missing MODIS Terra files 
# and replace them with the corresponding MODIS Aqua files
# 3. Use the script SnowCover_SnowCoverDuration.ipynb to produce datasets with information about the snow cover on the ground
# and the snow cover duration for each hydrological year and each pixel within the study region

In [ ]:
# load all the modules needed
import os
import shutil
import xarray
import h5py
import glob
import sys
import rasterio
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd

In [ ]:
# define needed folders, files and variables

# folders:
SCD_output_folder = 'path/to/your/SnowCoverDuration_folder/'

# files:
SRTM = 'path/to/h23v04_srtm_subset_corrected.tif'

# one MODIS Terra file to extract projection information from
terra_file = 'path/to/your/MODIS_file.hdf'
SCD_mean_output_file = 'SCD_mean.tif'

# variables:
elevation_threshold = 250

# projection information to save .tif files
hdf_ds = gdal.Open(terra_file, gdal.GA_ReadOnly)
ds = gdal.Open(hdf_ds.GetSubDatasets()[0][0], gdal.GA_ReadOnly)
proj_info = ds.GetGeoTransform(), ds.GetProjection()

In [ ]:
# functions:

# read in a geotiff file using GDAL
def read_tiff(file):
    ds = gdal.Open(file, gdal.GA_ReadOnly)
    band = ds.GetRasterBand(1)
    arr = band.ReadAsArray()
    return arr

# write geotiffs using GDAL (commented out the compresion because compressed files can not be read in later)
def write_tiff(outfile, data, proj_info, dtype=gdal.GDT_UInt16):
    x,y = data.shape
    dst = gdal.GetDriverByName('GTiff').Create(outfile,y,x,1,dtype)#options=['COMPRESS=DEFLATE']
    dst.SetGeoTransform(proj_info[0])
    dst.SetProjection(proj_info[1])
    dst.GetRasterBand(1).WriteArray(data)
    dst = None
    return ''

In [ ]:
# TASK: Plot of SCD for each 250-meter elevation zone

# calculate the mean SCD values for every hydrological year and for every 250-meter elevation zone

# list with the path to the SCD output files for every hydrological year
SCD_files = []

for filename in os.listdir(SCD_output_folder):
     if filename.endswith('tif'):
        SCD_file = os.path.join(SCD_output_folder, filename)
        SCD_files.append(SCD_file)

# create an empty dataframe to store the elevation and mean SCD values
df_mean_values = pd.DataFrame(columns = ['Year', 'Min_Elevation', 'Max_Elevation', 'Mean_Value'])

# load and read SRTM .tif file
srtm = read_tiff(SRTM)

# a few pixels are wrongly assigned to an elevation of over 32,000 m
# set the maximum elevation to 7500 m and assign all pixels above this values to 7500 m
srtm[srtm > 7000] = 7000

# calculate the number of 250m zones by dividing the maximum elevation by the threshold
num_zones = int(np.ceil(srtm.max() / elevation_threshold))

# loop through each elevation zone and extract the data for each year in the stack
for zone in range(num_zones):
    min_elevation = zone * elevation_threshold
    max_elevation = (zone + 1) * elevation_threshold

    # create a mask for the current elevation zone
    zone_mask = np.logical_and(srtm >= min_elevation, srtm < max_elevation)

    # loop through each raster file in the stack
    for raster_file in SCD_files:
        
        # open the raster dataset
        with rasterio.open(raster_file) as raster_dataset:
            # read the raster data for the current hydrological year as array
            year_data = raster_dataset.read(1).astype(np.float32)

            # apply the mask to extract the data for the current elevation zone and year
            zone_year_data = year_data.copy()
            zone_year_data[~zone_mask] = np.nan
            
            # calculate the mean of the data for the current year in the current elevation zone
            year_mean = np.nanmean(zone_year_data)

            # append the data to the data frame
            df_mean_values = df_mean_values.append({
                'Year': raster_file.split('_')[1].replace('.tif', ''),
                'Min_Elevation': min_elevation,
                'Max_Elevation': max_elevation,
                'Mean_Value': year_mean
            }, ignore_index = True)

In [ ]:
df_mean_values

In [ ]:
# line plot of the mean SCD for every hydrological year and every 250-meter elevation zone

# group the data frame by 'Min_Elevation' and 'Max_Elevation'
grouped = df_mean_values.groupby(['Min_Elevation', 'Max_Elevation'])
plt.figure(figsize=(10, 6))

# get the colormap based on the number of zones
cmap = plt.get_cmap('ocean', num_zones)

for name, group in grouped:
    # get the mean values and corresponding elevations for the current zone group
    elevations = (name[0], name[1])
    mean_values = group['Mean_Value']

    # plot the line with the corresponding color based on the zone index
    zone_index = int(name[0] / elevation_threshold)  # use the min elevation as the index for the zone
    plt.plot(group['Year'], mean_values, label=f'{elevations[0]}-{elevations[1]}', color=cmap(zone_index))

# add axis labels, title, and legend
plt.xlabel('Hydrological Year')
plt.ylabel('Mean Snow Cover Duration [days]')
plt.title('Mean SCD for each 250-meter Elevation Zone')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
# TASK: Determine the year with the highest and the lowest mean SCD for pixels above 400 meters

# open the SRTM file using rasterio
with rasterio.open(SRTM) as srtm_dataset:
    # read the elevation data as array
    srtm = srtm_dataset.read(1)

# create a mask for elevations above 400 meters
mask_above_400m = srtm > 400

# initialize a dataframe to store the mean values for elevations above 400 meters
mean_values_df = pd.DataFrame(columns = ['Year', 'Mean_Above_400m'])

# loop through each raster file
for SCD_file in SCD_files:
    with rasterio.open(os.path.join(SCD_output_folder, SCD_file)) as raster_dataset:
        SCD_data = raster_dataset.read(1)

    # apply the mask to the SCD raster data to extract values above 400 meters
    masked_SCD_data = np.ma.masked_array(SCD_data, mask=~mask_above_400m)
    
    # calculate the mean of values above 400 meters for this layer
    mean_above_400m = np.mean(masked_SCD_data)
    
    # append the data to the dataframe
    mean_values_df = mean_values_df.append({
        'Year': SCD_file.split('_')[1].replace('.tif', ''),
        'Mean_Above_400m': mean_above_400m
    }, ignore_index = True)
    

# plot the mean values for elevations above 400 meter and every hydrological year
plt.figure(figsize=(10, 6))

plt.plot(mean_values_df['Year'], mean_values_df['Mean_Above_400m'], color = 'darkblue', marker = 'o', markersize = 5)

plt.xlabel('Hydrological Year')
plt.ylabel('SCD Mean above 400m [days]')
plt.title('Mean SCD above 400 Meters')
plt.xticks()
plt.tight_layout()

plt.show()   

# print the data frame to determine the exact values
print(mean_values_df)

In [ ]:
# TASK: Mean SCD over the whole time series

# stack the SCD files into a 3D array
SCD_stacked = []

for SCD_file in SCD_files:
    with rasterio.open(SCD_file) as dataset:
        SCD_data = dataset.read(1)
        SCD_stacked.append(SCD_data)

SCD_stacked = np.array(SCD_stacked)

# calculate the mean values over all the rasters for each pixel
SCD_mean = np.nanmean(SCD_stacked, axis=0)

# save the array with mean values as a .tif file
# to be able to plot it later in QGIS
write_tiff(os.path.join(SCD_output_folder, SCD_mean_output_file), SCD_mean, proj_info)

# plot showing the mean snow cover duration over all the hydrological years
plt.imshow(SCD_mean)
plt.title('Mean Snow Cover Duration')
plt.colorbar()

plt.show()